# ML LAB_1 Feature Enginerring

Import required libraries

In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score

Load data from csv files

In [2]:

train_df = pd.read_csv("ML/train.csv")
valid_df= pd.read_csv("ML/valid.csv")
test_df = pd.read_csv("ML/test.csv")

Create a dictionary to store values related to each label

In [3]:
labels = ['label_1', 'label_2', 'label_3', 'label_4']
X_train = {}
X_valid = {}
y_train = {}
y_valid = {}
X_test = {}

Seperating feature data for each label

In [4]:
for label in labels:
    scaler = StandardScaler()
    tr_df = train_df
    val_df = valid_df
    tst_df = test_df
    if label == 'label_2': # Remove NaN rows for label_2
      tr_df = train_df[train_df[label].notna()]
      val_df = valid_df[valid_df[label].notna()]

    X_train[label] = pd.DataFrame(scaler.fit_transform(tr_df.iloc[:, :-4]))
    X_valid[label] = pd.DataFrame(scaler.transform(val_df.iloc[:, :-4]))
    X_test[label] = pd.DataFrame(scaler.transform(tst_df.iloc[:, :-4]))

    y_train[label] = tr_df[label]
    y_valid[label] = val_df[label]

In [5]:
X_train['label_1']

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,-1.719486,0.323086,1.226266,1.388683,0.664613,-0.308695,-0.812392,-0.655816,0.179515,0.140254,...,2.113138,1.527554,-0.982967,0.855748,-0.670494,-1.234800,-1.324892,-0.017108,-1.003996,0.938640
1,-1.497857,-0.390408,1.087442,3.184804,-0.932385,2.113584,0.386127,-0.558954,1.503739,-1.073030,...,1.368482,-0.911968,-2.100238,-0.073337,-0.646944,-0.706550,-1.624288,-0.387415,0.937548,-0.919349
2,-1.696629,0.257429,-0.436794,3.455455,-1.191656,1.396831,1.649450,-2.059777,1.294943,-1.389119,...,0.363484,0.440210,-2.993112,-0.249982,0.001384,0.122782,1.054587,0.770736,-1.294388,-0.823439
3,0.932099,-1.593295,-0.334127,0.685190,0.269621,1.007221,1.233836,0.237993,2.078308,0.407337,...,1.087123,-0.112537,-3.350270,-0.846158,0.744967,0.439564,-0.690871,0.543678,-1.567913,0.196262
4,-1.768010,0.119449,0.540221,2.725170,-0.285790,1.467148,-0.317170,0.131942,0.732225,-2.162524,...,1.398393,-0.132943,-1.811507,0.497713,-0.577680,-1.125974,0.150025,-0.989360,2.509757,0.165997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28515,0.093076,1.607867,-0.201254,-0.017915,3.488325,-0.084209,-1.277312,0.247766,1.599102,0.786603,...,0.870184,-0.189773,-0.642649,1.728059,0.776254,-1.112233,-0.875752,1.217201,0.705901,0.498509
28516,1.663375,-0.711944,0.061733,-0.060080,0.384315,-0.358914,1.009997,-2.121422,-0.189213,0.138299,...,0.884920,-0.563606,0.004032,1.218849,-0.162391,0.056092,-0.136669,-0.365558,1.144407,-0.433992
28517,1.371444,-0.016175,0.146018,0.348439,1.420678,0.295186,0.320466,-0.038330,0.285876,0.876290,...,0.412954,-0.219770,-0.461239,-1.311420,1.158437,-2.007156,1.070300,1.827668,-0.329699,-0.336336
28518,-0.776721,-0.005011,0.006696,-0.817254,1.184843,-0.796913,0.179448,-1.120960,0.272548,-0.995704,...,0.591453,0.445483,0.912035,0.320850,0.740111,-0.623250,0.172330,-0.197340,-1.414820,-0.166887


Feature Selection using K Best



In [5]:
new_features = 95
selector = SelectKBest(f_classif, k=new_features)

In [6]:
X_train_selected = {}
X_valid_selected = {}
X_test_selected = {}
for label in X_train:
  X_train_selected[label] = pd.DataFrame(selector.fit_transform(X_train[label], y_train[label]))
  X_valid_selected[label] = pd.DataFrame(selector.transform(X_valid[label]))
  X_test_selected[label] = pd.DataFrame(selector.transform(X_test[label]))

Check selected features

In [8]:
X_train_selected['label_1'].shape

(28520, 95)

Suppport Vector Machine Model

In [7]:
def support_vector_machine(L, training_data, validation_data, training_labels, validation_labels):
    if L == 'label_2': #for Age
        linear_regressor = LinearRegression()
        linear_regressor.fit(training_data[L], training_labels[L])
        predictions = linear_regressor.predict(validation_data[L])
        print(f"+ Mean Squared Error for {L} = ", mean_squared_error(validation_labels[L], predictions))
    else:
        svm_classifier = SVC(kernel='linear')
        svm_classifier.fit(training_data[L], training_labels[L])
        predictions = svm_classifier.predict(validation_data[L])
        print(f"+ Accuracy Score for {L} = ", accuracy_score(validation_labels[L], predictions))


Testing original data using evaluation metrices

In [8]:
support_vector_machine('label_1', X_train, X_valid, y_train, y_valid)
support_vector_machine('label_2', X_train, X_valid, y_train, y_valid)
support_vector_machine('label_3', X_train, X_valid, y_train, y_valid)
support_vector_machine('label_4', X_train, X_valid, y_train, y_valid)

+ Accuracy Score for label_1 =  0.9906666666666667
+ Mean Squared Error for label_2 =  23.82559204040575
+ Accuracy Score for label_3 =  0.9986666666666667
+ Accuracy Score for label_4 =  0.9626666666666667


In [11]:
X_train['label_1'].shape

(28520, 256)

In [12]:
X_train_selected['label_1'].shape

(28520, 95)

Evaluation after feature selection

In [9]:
support_vector_machine('label_1', X_train_selected, X_valid_selected, y_train, y_valid)
support_vector_machine('label_2', X_train_selected, X_valid_selected, y_train, y_valid)
support_vector_machine('label_3', X_train_selected, X_valid_selected, y_train, y_valid)
support_vector_machine('label_4', X_train_selected, X_valid_selected, y_train, y_valid)

+ Accuracy Score for label_1 =  0.9746666666666667
+ Mean Squared Error for label_2 =  28.883154091511987
+ Accuracy Score for label_3 =  0.9986666666666667
+ Accuracy Score for label_4 =  0.9133333333333333


Predict Labels

In [10]:
def svm_test(L, training_data, validation_data, training_labels, validation_labels):
    if L == 'label_2':
        svm_regressor = LinearRegression()
        svm_regressor.fit(training_data[L], training_labels[L])
        predictions = svm_regressor.predict(validation_data[L])
    else:
        svm_classifier = SVC(kernel='linear')
        svm_classifier.fit(training_data[L], training_labels[L])
        predictions = svm_classifier.predict(validation_data[L])
    return pd.DataFrame(predictions)

In [11]:
y_pred_l1 = svm_test('label_1', X_train, X_test, y_train, y_valid)
y_pred_l2 = svm_test('label_2', X_train, X_test, y_train, y_valid)
y_pred_l3 = svm_test('label_3', X_train, X_test, y_train, y_valid)
y_pred_l4 = svm_test('label_4', X_train, X_test, y_train, y_valid)

In [16]:
y_pred_l1

,0
0,45
1,45
2,45
3,45
4,45
...,...
745,39
746,39
747,39
748,39


In [ ]:
y_pred_selected_l1 = svm_test('label_1', X_train_selected, X_test_selected, y_train, y_valid)
y_pred_selected_l2 = svm_test('label_2', X_train_selected, X_test_selected, y_train, y_valid)
y_pred_selected_l3 = svm_test('label_3', X_train_selected, X_test_selected, y_train, y_valid)
y_pred_selected_l4 = svm_test('label_4', X_train_selected, X_test_selected, y_train, y_valid)

In [18]:
y_pred_selected_l1

,0
0,45
1,45
2,45
3,45
4,45
...,...
745,39
746,39
747,39
748,39


CSV file creation

In [ ]:
# Concatenate the DataFrames horizontally
new_features_str = str(new_features)
new_features_df = pd.DataFrame([new_features_str] * len(y_pred_l1))

In [ ]:
cn = {i: f'new_feature_{i+1}' for i in range(new_features)}

In [132]:
ncn = [f'new_feature_{i}' for i in range(new_features+1, 257)]
empty_columns = pd.DataFrame(columns=ncn)

In [ ]:
labels = ['label_1', 'label_2', 'label_3', 'label_4']

for label in labels:
    # Concatenate dataframes
    result_df = pd.concat([globals()[f'y_pred_l{label[-1]}'], globals()[f'y_pred_selected_l{label[-1]}'], new_features_df, pd.DataFrame(X_test_selected[label])], axis=1)
    
    # Define column names
    column_names = ['Predicted labels before feature engineering', 'Predicted labels after feature engineering', 'No of new features'] + list(pd.DataFrame(X_test_selected[label]))
    result_df.columns = column_names
    
    # Define the output file name
    output_file = f"ML/190705B_{label}.csv"
    
    # Save the dataframe to a CSV file
    result_df.to_csv(output_file, index=False)
